# Collaborative Filtering Based Recommendation System

## Import the Dependecies

In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from mysql.connector import MySQLConnection

## Establish the connection to MySQL Database

In [2]:
db = MySQLConnection(host = 'localhost',
                     username = 'root', 
                     password = 'mysqlNik631197',
                     database = 'books_database')

In [3]:
cursor = db.cursor(buffered= True)

## Importing the Dataset from Database and Preprocessing

In [4]:
cols = ['ISBN',
        'Book-Title',
        'Book-Author',
        'Year-Of-Publication',
        'Publisher',
        'Image-URL-S',
        'Image-URL-M',
        'Image-URL-L']

In [5]:
cursor.execute("SELECT * FROM books;")
books = cursor.fetchall()

In [6]:
df = pd.DataFrame(books, columns = cols)

In [7]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...
1,60542128,When the Storm Breaks,Heather Lowell,2003,HarperTorch,http://images.amazon.com/images/P/0060542128.0...,http://images.amazon.com/images/P/0060542128.0...,http://images.amazon.com/images/P/0060542128.0...
2,006092988x,A Tree Grows in Brooklyn,Betty Smith,1998,Perennial,http://images.amazon.com/images/P/006092988X.0...,http://images.amazon.com/images/P/006092988X.0...,http://images.amazon.com/images/P/006092988X.0...
3,60930535,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...,http://images.amazon.com/images/P/0060930535.0...
4,60934417,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...,http://images.amazon.com/images/P/0060934417.0...


In [8]:
df.shape

(1581, 8)

In [9]:
cols_rating = ['User_ID', 
               'ISBN', 
               'Rating']

In [10]:
cursor.execute("SELECT * FROM ratings")
rat = pd.DataFrame(cursor.fetchall(), columns = cols_rating)

In [11]:
rat.head()

,User_ID,ISBN,Rating
0,277427,002542730x,10
1,3363,002542730x,0
2,11676,002542730x,6
3,12538,002542730x,10
4,13552,002542730x,0


In [12]:
df = df.merge(rat, on = 'ISBN')

In [13]:
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User_ID,Rating
0,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,277427,10
1,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,3363,0
2,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,11676,6
3,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,12538,10
4,002542730x,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,http://images.amazon.com/images/P/002542730X.0...,13552,0


In [14]:
df1 = df[['User_ID', 'Book-Title', 'Rating']].copy()

In [15]:
df1.head()

,User_ID,Book-Title,Rating
0,277427,Politically Correct Bedtime Stories: Modern Ta...,10
1,3363,Politically Correct Bedtime Stories: Modern Ta...,0
2,11676,Politically Correct Bedtime Stories: Modern Ta...,6
3,12538,Politically Correct Bedtime Stories: Modern Ta...,10
4,13552,Politically Correct Bedtime Stories: Modern Ta...,0


In [16]:
user = list(df1['User_ID'].unique())
len(user)

886

In [17]:
df['ISBN'].value_counts()

971880107     363
051513628x    272
316666343     270
60928336      220
440214041     218
             ... 
671016768      25
1896860982     25
140265686      25
671525433      25
60549270       25
Name: ISBN, Length: 1580, dtype: int64

In [18]:
d = dict()
for i in user:
    temp = df1[df1['User_ID']==i].copy()
    book = list(temp['Book-Title'])
    ratings = list(temp['Rating'])
    t = dict(zip(book, ratings))
    d[i] = t

In [19]:
d

{'277427': {'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times': 10,
  'When the Storm Breaks': 7,
  'A Tree Grows in Brooklyn': 0,
  'The Poisonwood Bible: A Novel': 0,
  'Bel Canto: A Novel': 0,
  'One for the Money (Stephanie Plum Novels (Paperback))': 9,
  'The Tao of Pooh': 0,
  'The Secret Life of Bees': 0,
  'The Purpose-Driven Life: What on Earth Am I Here For?': 0,
  'Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)': 0,
  "Full Tilt (Janet Evanovich's Full Series)": 0,
  'Lucky : A Memoir': 0,
  'The Dogs of Babel (Today Show Book Club #12)': 0,
  "White Oleander : A Novel (Oprah's Book Club)": 0,
  'The Jester': 0,
  'The Lovely Bones: A Novel': 0,
  'Me Talk Pretty One Day': 8,
  'Naked': 0,
  'Private Screening': 0,
  'Escape the Night': 0,
  'The Hobbit : The Enchanting Prelude to The Lord of the Rings': 0,
  'A Prayer for Owen Meany': 0,
  'Silent Witness': 0,
  'No Safe Place': 0,
  'The Last Don': 0,
  'The Murder Book': 10,


In [20]:
df_final = pd.DataFrame(d.values())

In [21]:
df_final['User_ID'] = d.keys()

In [22]:
df_final

,Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,When the Storm Breaks,A Tree Grows in Brooklyn,The Poisonwood Bible: A Novel,Bel Canto: A Novel,One for the Money (Stephanie Plum Novels (Paperback)),The Tao of Pooh,The Secret Life of Bees,The Purpose-Driven Life: What on Earth Am I Here For?,Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel),...,This Matter Of Marriage,Moving Target,The Scottish Bride (Bride Trilogy (Paperback)),Until You,Lawless,Perfect,Hellion Bride (Bride Trilogy (Paperback)),Drums of Autumn,Snagged,User_ID
0,10.0,7.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,277427
1,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3363
2,6.0,8.0,9.0,0.0,0.0,8.0,8.0,0.0,NaN,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11676
3,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12538
4,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155916
882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35433
883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163319
884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142454


In [23]:
df_final.fillna(0, inplace = True)

In [24]:
df_final.head()

,Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,When the Storm Breaks,A Tree Grows in Brooklyn,The Poisonwood Bible: A Novel,Bel Canto: A Novel,One for the Money (Stephanie Plum Novels (Paperback)),The Tao of Pooh,The Secret Life of Bees,The Purpose-Driven Life: What on Earth Am I Here For?,Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel),...,This Matter Of Marriage,Moving Target,The Scottish Bride (Bride Trilogy (Paperback)),Until You,Lawless,Perfect,Hellion Bride (Bride Trilogy (Paperback)),Drums of Autumn,Snagged,User_ID
0,10.0,7.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277427
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3363
2,6.0,8.0,9.0,0.0,0.0,8.0,8.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11676
3,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12538
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13552


In [25]:
import pickle

pickle.dump(df_final, open("User_Ratings.pkl", "wb"))

In [26]:
df_final.set_index("User_ID", inplace = True)

In [27]:
df_final.head()

,Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,When the Storm Breaks,A Tree Grows in Brooklyn,The Poisonwood Bible: A Novel,Bel Canto: A Novel,One for the Money (Stephanie Plum Novels (Paperback)),The Tao of Pooh,The Secret Life of Bees,The Purpose-Driven Life: What on Earth Am I Here For?,Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel),...,The Sky Is Falling,This Matter Of Marriage,Moving Target,The Scottish Bride (Bride Trilogy (Paperback)),Until You,Lawless,Perfect,Hellion Bride (Bride Trilogy (Paperback)),Drums of Autumn,Snagged
User_ID,,,,,,,,,,,,,,,,,,,,,
277427,10.0,7.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11676,6.0,8.0,9.0,0.0,0.0,8.0,8.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12538,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
def normalize(row):
    new_row = (row - row.mean())/(row.max() - row.min())
    return new_row

df_std = df_final.apply(normalize)

In [30]:
df_std.fillna(0, inplace = True)

## User Based Collaborative Filtering

In [31]:
user_similarity = cosine_similarity(df_std)

In [32]:
user_similarity

array([[ 1.        , -0.01861811,  0.03791697, ..., -0.01614664,
        -0.03977074, -0.03977074],
       [-0.01861811,  1.        , -0.03316738, ...,  0.01347467,
         0.05303765,  0.05303765],
       [ 0.03791697, -0.03316738,  1.        , ..., -0.17188677,
        -0.58098114, -0.58098114],
       ...,
       [-0.01614664,  0.01347467, -0.17188677, ...,  1.        ,
         0.30614789,  0.30614789],
       [-0.03977074,  0.05303765, -0.58098114, ...,  0.30614789,
         1.        ,  1.        ],
       [-0.03977074,  0.05303765, -0.58098114, ...,  0.30614789,
         1.        ,  1.        ]])

In [33]:
user_sim_df = pd.DataFrame(user_similarity, index = df_std.index, columns = df_std.index)
user_sim_df.head()

User_ID,277427,3363,11676,12538,13552,16795,24194,25981,26535,28204,...,133868,222488,151806,222204,261037,155916,35433,163319,142454,193560
User_ID,,,,,,,,,,,,,,,,,,,,,
277427,1.000000,-0.018618,0.037917,0.035631,0.002787,-0.012910,0.042238,-0.023745,-0.019122,0.048003,...,-0.039771,-0.016360,-0.020981,-0.014449,-0.021146,-0.039771,-0.039771,-0.016147,-0.039771,-0.039771
3363,-0.018618,1.000000,-0.033167,0.048277,-0.037617,-0.037118,-0.007568,-0.021647,0.013129,-0.017908,...,0.053038,0.004553,0.025332,0.008410,0.018415,0.053038,0.053038,0.013475,0.053038,0.053038
11676,0.037917,-0.033167,1.000000,0.002173,0.116645,0.238499,-0.093848,0.063608,-0.201950,-0.025963,...,-0.580981,-0.129873,-0.292559,-0.117529,-0.235074,-0.580981,-0.580981,-0.171887,-0.580981,-0.580981
12538,0.035631,0.048277,0.002173,1.000000,-0.037035,0.077689,-0.020588,-0.000921,-0.013951,-0.024747,...,-0.023126,-0.013465,-0.012806,-0.010888,-0.014536,-0.023126,-0.023126,-0.011234,-0.023126,-0.023126
13552,0.002787,-0.037617,0.116645,-0.037035,1.000000,0.075548,-0.046535,0.047623,-0.072331,-0.016975,...,-0.189778,-0.051669,-0.096069,-0.052345,-0.085944,-0.189778,-0.189778,-0.064727,-0.189778,-0.189778


In [34]:
pickle.dump(user_sim_df, open("user_similarity_df.pkl", "wb"))

## Item Based Collaborative Filtering

In [35]:
item_similarity = cosine_similarity(df_std.T)

In [36]:
item_sim_df = pd.DataFrame(item_similarity, index = df_std.columns, columns = df_std.columns)
item_sim_df.head()

,Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,When the Storm Breaks,A Tree Grows in Brooklyn,The Poisonwood Bible: A Novel,Bel Canto: A Novel,One for the Money (Stephanie Plum Novels (Paperback)),The Tao of Pooh,The Secret Life of Bees,The Purpose-Driven Life: What on Earth Am I Here For?,Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel),...,The Sky Is Falling,This Matter Of Marriage,Moving Target,The Scottish Bride (Bride Trilogy (Paperback)),Until You,Lawless,Perfect,Hellion Bride (Bride Trilogy (Paperback)),Drums of Autumn,Snagged
Politically Correct Bedtime Stories: Modern Tales for Our Life and Times,1.000000,0.116153,0.040857,0.003998,0.037906,0.052073,0.102874,-0.034878,-0.013465,0.031808,...,-0.007421,-0.011522,-0.013543,-0.016266,-0.014483,-0.012531,-0.015332,-0.010477,-0.017440,-0.010339
When the Storm Breaks,0.116153,1.000000,0.093715,-0.017268,-0.021282,0.127328,0.045850,0.090731,-0.009049,0.109261,...,-0.004987,0.106235,-0.009101,-0.010931,0.324792,0.047795,0.159635,-0.007041,-0.011720,-0.006948
A Tree Grows in Brooklyn,0.040857,0.093715,1.000000,0.060480,-0.019313,0.036690,0.049575,-0.021271,-0.008212,0.037491,...,-0.004526,-0.007027,-0.008260,-0.009920,-0.008833,-0.007642,-0.009351,-0.006390,0.075490,-0.006306
The Poisonwood Bible: A Novel,0.003998,-0.017268,0.060480,1.000000,0.083409,0.088506,0.005747,0.019684,-0.014151,-0.017680,...,0.095666,-0.012109,0.008246,0.114659,-0.015222,-0.013169,-0.016114,-0.011011,-0.018329,-0.010866
Bel Canto: A Novel,0.037906,-0.021282,-0.019313,0.083409,1.000000,0.127166,0.002982,0.090034,-0.017441,0.017555,...,-0.009612,-0.014924,-0.017542,-0.021069,-0.018760,-0.016231,-0.019860,-0.013570,0.102914,-0.013392


In [37]:
pickle.dump(item_sim_df, open("item_similarity_df.pkl", "wb"))

In [48]:
def get_similar_books(book_name, user_rating):
    sim = item_sim_df[book_name] * (user_rating - 5)
    sim = sim.sort_values(ascending = False)
    
    return sim[1:6]

In [49]:
print(get_similar_books("When the Storm Breaks", 7))

Dawn in Eclipse Bay    1.182015
Forbidden Fruit        1.029969
Impulse                0.969344
The Playboy            0.947884
Head Over Heels        0.931603
Name: When the Storm Breaks, dtype: float64


In [42]:
sample = [("Politically Correct Bedtime Stories: Modern Tales for Our Life and Times", 8), 
          ("The Poisonwood Bible: A Novel", 9), 
          ("The Pelican Brief", 4),
          ("Harry Potter and the Prisoner of Azkaban (Book 3)", 10),
          ("The Playboy", 3)]

In [44]:
sim_books = pd.DataFrame()

for book, rating in sample:
    sim_books = sim_books.append(get_similar_books(book, rating), ignore_index = True)
    
sim_books.head()

,16 Lighthouse Road,1984,1st to Die: A Novel,2010: Odyssey Two,204 Rosewood Lane,24 Hours,2nd Chance,3rd Degree,4 Blondes,A 2nd Helping of Chicken Soup for the Soul (Chicken Soup for the Soul Series (Paper)),...,"Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",Writ of Execution,Wuthering Heights,Year of Wonders,Year of Wonders: A Novel of the Plague,You Belong To Me,You Belong to Me and Other True Cases (Ann Rule's Crime Files: Vol. 2),"You Can't Scare Me! (Goosebumps, No 15)",Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya
0,-0.041019,0.066313,0.097187,-0.046485,-0.043855,0.564917,0.182381,0.043127,-0.054531,-0.049536,...,0.274202,0.186820,-0.041372,0.143160,0.127400,0.066420,-0.027203,0.425558,0.060164,0.123542
1,0.132525,-0.025235,-0.012945,-0.065139,-0.061454,-0.053934,-0.092770,-0.060753,0.159705,-0.069415,...,-0.096126,-0.056785,-0.057975,0.388799,-0.062195,0.217735,-0.038120,-0.030060,0.362801,-0.059448
2,-0.165892,-0.024784,-0.077616,-0.130070,0.023267,-0.082291,-0.118455,0.000116,0.006334,-0.104092,...,0.005502,-0.099713,0.021950,0.039380,0.023547,-0.098222,-0.276686,-0.049954,-0.022783,-0.113012
3,-0.094307,0.344561,-0.100845,-0.106872,0.175850,0.211465,0.011084,-0.099677,-0.125372,-0.113888,...,0.031628,-0.093166,-0.095118,0.004031,0.263647,-0.118551,-0.062542,-0.049319,0.067505,-0.097535
4,-0.427545,0.023645,-0.334603,0.014766,-0.588875,0.012226,-0.460101,-0.563827,0.017322,0.015735,...,-0.225722,-0.493256,0.013142,0.023578,-0.368338,-0.246285,0.008641,-0.988351,-0.245870,-0.356008


In [51]:
sim_books = sim_books.sum().sort_values(ascending = False)

In [52]:
sim_books

Harry Potter and the Prisoner of Azkaban (Book 3)                           5.103663
The Poisonwood Bible: A Novel                                               3.815291
Politically Correct Bedtime Stories: Modern Tales for Our Life and Times    3.238954
Harry Potter and the Goblet of Fire (Book 4)                                3.031627
Harry Potter and the Sorcerer's Stone (Book 1)                              2.407791
                                                                              ...   
Face the Fire (Three Sisters Island Trilogy)                               -1.076508
Cold Blooded                                                               -1.209466
Purity in Death                                                            -1.214504
Like Silk                                                                  -1.349264
The Playboy                                                                -1.855260
Length: 1581, dtype: float64

In [54]:
sample

[('Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
  8),
 ('The Poisonwood Bible: A Novel', 9),
 ('The Pelican Brief', 4),
 ('Harry Potter and the Prisoner of Azkaban (Book 3)', 10),
 ('The Playboy', 3)]

In [59]:
fav = list(zip(*sample))[0]

In [60]:
fav

('Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
 'The Poisonwood Bible: A Novel',
 'The Pelican Brief',
 'Harry Potter and the Prisoner of Azkaban (Book 3)',
 'The Playboy')

In [63]:
recommend = []
count = 0
for i in dict(sim_books).keys():
    if i not in fav:
        recommend.append(i)
        count+=1
    if count == 5:
        break

In [64]:
recommend

['Harry Potter and the Goblet of Fire (Book 4)',
 "Harry Potter and the Sorcerer's Stone (Book 1)",
 'Harry Potter and the Order of the Phoenix (Book 5)',
 'The Tao of Pooh',
 'Stuart Little']

In [65]:
sim_books[:10]

Harry Potter and the Prisoner of Azkaban (Book 3)                           5.103663
The Poisonwood Bible: A Novel                                               3.815291
Politically Correct Bedtime Stories: Modern Tales for Our Life and Times    3.238954
Harry Potter and the Goblet of Fire (Book 4)                                3.031627
Harry Potter and the Sorcerer's Stone (Book 1)                              2.407791
Harry Potter and the Order of the Phoenix (Book 5)                          1.805074
The Tao of Pooh                                                             1.229580
Stuart Little                                                               1.149012
Mrs Dalloway                                                                1.106067
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))            1.097626
dtype: float64